In [1]:
# Import Packages
import numpy as np
import pandas as pd

from PIL import Image
import cv2

import matplotlib.pyplot as plot
import matplotlib.patches as patches

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import h5py

In [7]:
# Mask Detection Function
def MaskDetectionVideo(img,classifier=cv2.CascadeClassifier("../Data/processed/classifier.xml"),model=keras.models.load_model("../Data/processed/model.h5"),sf=1.1,mn=1):
    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    detections = classifier.detectMultiScale(img,scaleFactor=sf,minNeighbors=mn)
    for detection in detections:
        xmin = int(detection[0])
        ymin = int(detection[1])
        width = int(detection[2])
        height = int(detection[3])
        xmax = xmin+width
        ymax = ymin+height
        face = img[ymin:ymax, xmin:xmax]
        resized = cv2.resize(face,(32,32))
        scaledInput = resized/255.0
        test = np.stack([scaledInput])
        prediction = model.predict_classes(test)
        color = [0,0,255] if prediction[0][0]==0 else [255,0,0]
        return cv2.rectangle(frame,(xmin,ymin),(xmax,ymax),color,2)
    

In [ ]:
# Video model prediction
# Not currently working (Likely due to video inability to capture video within Azure ML)
vid = cv2.VideoCapture(0) 
model=keras.models.load_model("../Data/processed/model.h5")
classifier=cv2.CascadeClassifier("../Data/processed/classifier.xml")  
while(True): 
    ret, frame = vid.read() 
    classified = MaskDetectionVideo(frame,classifier,model)
    cv2.imshow('frame', classified) 
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
vid.release() 
cv2.destroyAllWindows() 